# Exercício 01 - Titanic

Esse exercício tem como objetivo exercitar os conceitos básicos de dataset e de aprendizagem de máquina vistos no tutorial *HelloWorld* e prepara-lo para o que será abordado nas próximas aulas.

No exercício vamos trabalhar com um outro clássico dataset de aprendizagem de máquina: sobreviventes do titanic. Este também é um problema de classificação que visa dado um conjunto de informações dos passageiros do titanic, pretende-se classifica-los em "sobreviveu" ou "não sobreviveu" ao naufrágio. 

A sua tarefa principal é pesquisar sobre o KNN no scikit learn e aplica-lo a este problema.

A base não vem com o ScikitLearn e vamos precisar baixar e carrega-la para trabalhar neste exemplo.

Para baixar e deixar a base pronta para uso segui o tutorial a seguir: 

http://nbviewer.jupyter.org/github/savarin/pyconuk-introtutorial/blob/master/notebooks/Section%201-0%20-%20First%20Cut.ipynb

Como a base é da competição do Kaggle, não irei disponibiliza-la aqui. Para baixar os arquivos necessários, acesse: 

https://www.kaggle.com/c/titanic/data (é necessário fazer um cadastro no Kaggle)

## Preparando a base de dados

Esta etapa não é o foco do nosso exercício. Sendo assim, só irei reproduzir o passo a passo do link passado anteriormente.

In [9]:
import pandas as pd
import numpy as np

print("Tabela Original")
df.head()

Tabela Original


,Survived,PassengerId,Pclass,Age,SibSp,Parch,Fare,Gender,Port
0,0,1,3,22.0,1,0,7.2500,1,2
1,1,2,1,38.0,1,0,71.2833,0,1
2,1,3,3,26.0,0,0,7.9250,0,2
3,1,4,1,35.0,1,0,53.1000,0,2
4,0,5,3,35.0,0,0,8.0500,1,2


In [10]:
df = pd.read_csv('../datasets/train.csv')
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df = df.dropna()
df['Gender'] = df['Sex'].map({'female': 0, 'male':1}).astype(int)
df['Port'] = df['Embarked'].map({'C':1, 'S':2, 'Q':3}).astype(int)
df = df.drop(['Sex', 'Embarked'], axis=1)
cols = df.columns.tolist()
cols = [cols[1]] + cols[0:1] + cols[2:]
df = df[cols]


print("Tabela depois de processada")
df.head()

Tabela depois de processada


,Survived,PassengerId,Pclass,Age,SibSp,Parch,Fare,Gender,Port
0,0,1,3,22.0,1,0,7.2500,1,2
1,1,2,1,38.0,1,0,71.2833,0,1
2,1,3,3,26.0,0,0,7.9250,0,2
3,1,4,1,35.0,1,0,53.1000,0,2
4,0,5,3,35.0,0,0,8.0500,1,2


O mesmo processo deve ser feito com a base de teste.

In [12]:
df_test = pd.read_csv('../datasets/test.csv')
df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df_test = df_test.dropna()
df_test['Gender'] = df_test['Sex'].map({'female': 0, 'male':1})
df_test['Port'] = df_test['Embarked'].map({'C':1, 'S':2, 'Q':3})
df_test = df_test.drop(['Sex', 'Embarked'], axis=1)

df_test.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Gender,Port
0,892,3,34.5,0,0,7.8292,1,3
1,893,3,47.0,1,0,7.0000,0,2
2,894,2,62.0,0,0,9.6875,1,3
3,895,3,27.0,0,0,8.6625,1,2
4,896,3,22.0,1,1,12.2875,0,2


Por fim, geramos nossa base de treino e teste no formato para o ScikitLearn

In [22]:
dataset = {
    'data': df.values[0:,2:],
    'target': df.values[0:,0]
}

In [23]:
print(dataset['data'])

[[  3.      22.       1.     ...,   7.25     1.       2.    ]
 [  1.      38.       1.     ...,  71.2833   0.       1.    ]
 [  3.      26.       0.     ...,   7.925    0.       2.    ]
 ..., 
 [  1.      19.       0.     ...,  30.       0.       2.    ]
 [  1.      26.       0.     ...,  30.       1.       1.    ]
 [  3.      32.       0.     ...,   7.75     1.       3.    ]]


In [24]:
print(dataset['target'])

[ 0.  1.  1.  1.  0.  0.  0.  1.  1.  1.  1.  0.  0.  0.  1.  0.  0.  0.
  1.  1.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  1.  0.
  0.  0.  1.  1.  0.  1.  0.  1.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.
  0.  0.  1.  0.  1.  1.  0.  1.  0.  1.  1.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  1.  1.  0.  0.  0.  0.  1.  0.  0.  1.
  0.  0.  0.  1.  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.
  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
  0.  1.  1.  1.  0.  0.  1.  0.  1.  1.  1.  1.  0.  0.  0.  0.  0.  1.
  0.  0.  1.  1.  1.  0.  1.  0.  0.  1.  1.  0.  1.  0.  1.  0.  0.  1.
  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
  1.  1.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  0.  0.  0.  0.  1.  1.
  1.  1.  1.  0.  1.  0.  0.  1.  0.  0.  0.  1.  0.  1.  0.  1.  1.  1.
  1.  0.  0.  0.  0.  0.  1.  0.  1.  1.  0.  1.  1

In [25]:
X = dataset['data']
Y = dataset['target']

## Tarefas

Como o propósito desse exercício não é submeter para o Kaggle, vamos ter que criar nossa própria base de testes. A base de teste disponibilizada pelo kaggle não vem com o gabarito, logo não poderíamos utilizar o aprendizado supervisionado.

Sendo assim, as tarefas deste exercício são:

1. Separe sua base em treino e teste. Para o teste use 10% da base original. 
2. Aplique o algoritmo de Árvore de Decisão visto no *HelloWorld* e calcule a acurácia na base de teste
3. Pesquise sobre o KNN e aplique-o sobre a mesma base de dados e calcule a acurácia